In [1]:
from src.data_processing import data_processing, data_dropping
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, Lasso, LinearRegression
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, make_scorer, accuracy_score, mean_squared_error
from nltk.stem import WordNetLemmatizer as Lemmatizer
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
import re

In [3]:
with open('data/text.p', 'rb') as file:
    text = pickle.load(file)
with open('data/target.p', 'rb') as file:
    target = pickle.load(file)

In [4]:
def hasnumber(s):
    return bool(re.search(r'\d', s))

In [495]:
pred = data_processing(data_dropping(predictor))

In [496]:
pred = pred[['existing_stories', 'existing_use', 'proposed_use', 'description']]

In [14]:
pred['has_existing_stories'] = (pred['existing_stories'].fillna(0) > 0).astype(int)

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [497]:
pred[['existing_use', 'proposed_use', 'description']] = pred[['existing_use', 'proposed_use', 'description']].fillna('-').astype(str)

In [498]:
pred['text'] = pred[['existing_use', 'proposed_use', 'description']].apply(lambda x: ' '.join(x), axis = 1)

In [581]:
float('29')

29.0

In [474]:
with open('data/pred_trimmed.p', 'wb') as file:
    pickle.dump(pred, file)

In [603]:
text = pred['text'].values

In [8]:
with open('data/text.p', 'wb') as file:
    pickle.dump(text, file)

In [5]:
y_full = (target['fire_rate_after'] < target['fire_rate_before']).astype(int).values

In [6]:
def int_able(str):
    try:
        float(str)
    except:
        return False
    return True

In [7]:
text = np.array([' '.join([Lemmatizer.lemmatize('n', word.lower()) for word in doc.split() if not hasnumber(word)]) for doc in text])

In [607]:
text_lemmatized = text

In [11]:
n = 50000
sample_idx = np.random.choice(len(text), n)

In [12]:
X = text[sample_idx]
y = y_full[sample_idx]

In [13]:
train = np.full(n, False)
train_idx = np.random.choice(n, int(.75*n), replace = False)
for idx in train_idx:
    train[idx] = True
test = ~train

In [14]:
vec = TfidfVectorizer(stop_words = 'english')

In [15]:
X = vec.fit_transform(X)

In [16]:
model = LogisticRegression()

In [17]:
model.fit(X[train],y[train])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [18]:
model.score(X[test], y[test])

0.90536

In [19]:
roc_auc_score(y[test], model.predict_proba(X[test])[:,1])

0.7904914071327358

In [619]:
vocab_idx[13029]

'spinkler'

In [21]:
roc_auc_score(y[train], model.predict_proba(X[train])[:,1])

0.8557572106736399

In [22]:
def f1(y_test, y_pred_prob, threshold):
    y_pred = (y_pred_prob > threshold).astype(int)
    return f1_score(y_test, y_pred)

In [23]:
current_best = 0
best_i = 0
for i in range(1000):
    f = f1(y[test], model.predict_proba(X[test])[:,1], i/1000)
    if f > current_best:
        current_best = f
        best_i = i
    

C:\Users\pengj\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
f1(y[test], model.predict_proba(X[test])[:,1], best_i/1000)

0.4188846641318124

In [509]:
vec.vocabulary_;

In [668]:
vocab_idx = {v: k for k, v in vec.vocabulary_.items()}

In [669]:
important_words = {}

In [670]:
for i, coef in enumerate(model.coef_[0]):
    if coef > 0:
        important_words[vocab_idx[i]] = coef

In [671]:
c = Counter(important_words)

In [674]:
top_decrease_1

[('office', 2.1176786110528223),
 ('dated', 1.7203072459482143),
 ('carpet', 1.7090974645931714),
 ('life', 1.6938419110066842),
 ('corridor', 1.6058811823357426),
 ('bart', 1.5648986145276225),
 ('nan', 1.5508567940393705),
 ('flourescent', 1.4961213796842556),
 ('sale', 1.4889250957687659),
 ('school', 1.4739605915228873),
 ('el', 1.4540642121475595),
 ('fargo', 1.4456208844155471),
 ('nursing', 1.4384922900103676),
 ('speakers', 1.429166610456676),
 ('oti', 1.4217271795798598),
 ('finishesnan', 1.414208629330898),
 ('center', 1.3983382701292237),
 ('hotel', 1.380807279973869),
 ('energy', 1.3464640793875338),
 ('motion', 1.3119829231303992)]

In [681]:
top_increase_2

[('maher', 4.135960726805149),
 ('apt', 3.2571120896300103),
 ('traps', 2.529927485747275),
 ('ordinance', 2.5289745837396778),
 ('speaker', 2.271684087536629),
 ('egress', 2.095901081992529),
 ('ref', 2.062200879369185),
 ('mep', 2.044750346887388),
 ('exh', 2.032254322337327),
 ('damage', 2.0148565235091977),
 ('modification', 2.0038582407406706),
 ('deferred', 2.001935973309281),
 ('evacuation', 1.9743179655078928),
 ('reroof', 1.9415711275179024),
 ('fiberglass', 1.813847341062362),
 ('occupied', 1.7999530375657145),
 ('community', 1.7843406873482186),
 ('maintain', 1.7662882423747037),
 ('sleeping', 1.7617268526760557),
 ('priority', 1.7045146788140375)]

In [679]:
w1 = [x[0] for x in top_increase_1]
w2 = [x[0] for x in top_increase_2]
[w for w in w1 if w in w2]

['maher', 'traps', 'apt', 'mep', 'evacuation', 'occupied', 'ordinance']

In [381]:
important_words_4 = important_words

In [392]:
important_words;

In [415]:
top10_2 = c.most_common(10)

In [401]:
top10_1 = c.most_common(10)

In [433]:
top10_3

[('traps', 9.066135535601088),
 ('exh', 5.236541100818746),
 ('265', 4.812601390528742),
 ('maher', 4.730539830504769),
 ('tags', 4.297665411723898),
 ('geotech', 4.20148706744233),
 ('901', 4.104538395668117),
 ('9884', 4.058352578295828),
 ('notifier', 3.8099096402798533),
 ('144', 3.680255661760993)]

In [416]:
top10_2

[('traps', 7.005050918380217),
 ('maher', 4.692009685187687),
 ('forms', 3.636798151040242),
 ('ref', 3.2956199551363703),
 ('evacuation', 2.8559641655760863),
 ('apt', 2.763896991448432),
 ('assmbly', 2.759404362814315),
 ('3884sf', 2.631019912856813),
 ('suite', 2.5307707835609),
 ('unit', 2.5235550187500255)]

In [417]:
top10_1

[('traps', 6.190488019758687),
 ('maher', 4.546629916650752),
 ('uprights', 3.1050460402905613),
 ('daycare', 2.8857078892696255),
 ('sounders', 2.7342078862780332),
 ('altered', 2.622869978271953),
 ('evacuation', 2.6174622302659016),
 ('washer', 2.552283019923069),
 ('arch', 2.4326521113122124),
 ('chap', 2.366688547050922)]

In [675]:

for i, row in pred[pred['description'].apply(lambda x: ' FARGO ' in x)].reset_index().iterrows():
    print(row['description'])
    if i > 10:
        break

INSTALL (N) FIRE ALARM SYSTEM FOR WELLS FARGO BANK                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
RELOCATE 4 FIRE SPRINKLERS FOR MODIFIED CEILING IN WELLS FARGO IN STORE. DEMO 1 FIRE SPINKLERnan
FIRE SPRINKLER TENANT IMPROVEMENTS  TO 10TH FLR. WELLS FARGO EMPLOYEE RELATION. 59 RELOC. 19 ADDS TOTAL 78 

In [26]:
from sklearn.decomposition import NMF

In [27]:
factor = NMF(n_components = 5)

In [30]:
factor.fit_transform(X)

array([[0.00207972, 0.02929167, 0.00565341, 0.        , 0.        ],
       [0.03912327, 0.00177047, 0.00607075, 0.        , 0.00021564],
       [0.00344825, 0.04062147, 0.00125605, 0.        , 0.        ],
       ...,
       [0.00091693, 0.00162768, 0.03989695, 0.01737722, 0.        ],
       [0.03748331, 0.00639017, 0.0062129 , 0.        , 0.        ],
       [0.00676866, 0.03529878, 0.0029804 , 0.01484486, 0.        ]])

In [35]:
factor.components_.shape

(5, 15641)